In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time 
import csv

import gmaps.datasets

#from ipywidgets.embed import embed_minimal_html 


# Import API key
from api_keys import g_key

In [2]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = os.path.join("..","python-api-challenge","output_data", "cities.csv")

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Okhotsk,100,RU,1639975062,95,59.3833,143.3000,3.34,12.19
1,Lebu,27,CL,1639974883,94,-37.6167,-73.6500,56.43,14.18
2,Puerto Ayora,95,EC,1639975062,94,-0.7393,-90.3518,69.28,6.62
3,Vaini,75,TO,1639975063,70,-21.2000,-175.2000,84.36,9.22
4,Salalah,20,OM,1639975063,68,17.0151,54.0924,71.69,5.75
...,...,...,...,...,...,...,...,...,...
551,Korla,63,CN,1639975202,46,41.7597,86.1469,27.07,1.45
552,Xingcheng,4,CN,1639975202,31,40.6167,120.7167,46.99,4.90
553,Nurota,0,UZ,1639975203,49,40.5614,65.6886,42.89,3.06
554,Tamandaré,75,BR,1639975203,81,-8.7597,-35.1047,77.86,7.45


In [3]:
# Configure gmaps:
gmaps.configure(api_key=g_key)


In [4]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [5]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
8,Cape Town,0,ZA,1639975064,72,-33.9258,18.4232,70.11,1.01
21,Along,0,IN,1639975067,26,28.1667,94.7667,70.21,1.16
125,Três Lagoas,0,BR,1639974876,75,-20.7511,-51.6783,71.37,8.57
184,Sittwe,0,MM,1639974889,59,20.1500,92.9000,74.23,1.95
372,Iquitos,0,PE,1639975156,89,-3.7481,-73.2472,78.84,1.14
445,Yārāda,0,IN,1639975176,61,17.6500,83.2667,75.67,5.44
547,Porto Velho,0,BR,1639975201,94,-8.7619,-63.9039,75.25,0.00


In [7]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
8,Cape Town,0,ZA,1639975064,72,-33.9258,18.4232,70.11,1.01,The Bay Hotel
21,Along,0,IN,1639975067,26,28.1667,94.7667,70.21,1.16,Reyi Homestay
125,Três Lagoas,0,BR,1639974876,75,-20.7511,-51.6783,71.37,8.57,Taj Hotel
184,Sittwe,0,MM,1639974889,59,20.1500,92.9000,74.23,1.95,Royal Sittwe Resort
372,Iquitos,0,PE,1639975156,89,-3.7481,-73.2472,78.84,1.14,DoubleTree by Hilton Hotel Iquitos
445,Yārāda,0,IN,1639975176,61,17.6500,83.2667,75.67,5.44,Ginger Vishakhapatnam Gajuwaka
547,Porto Velho,0,BR,1639975201,94,-8.7619,-63.9039,75.25,0.00,Hotel Slaviero Essential Porto Velho


In [8]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
8,Cape Town,0,ZA,1639975064,72,-33.9258,18.4232,70.11,1.01,The Bay Hotel
21,Along,0,IN,1639975067,26,28.1667,94.7667,70.21,1.16,Reyi Homestay
125,Três Lagoas,0,BR,1639974876,75,-20.7511,-51.6783,71.37,8.57,Taj Hotel
184,Sittwe,0,MM,1639974889,59,20.1500,92.9000,74.23,1.95,Royal Sittwe Resort
372,Iquitos,0,PE,1639975156,89,-3.7481,-73.2472,78.84,1.14,DoubleTree by Hilton Hotel Iquitos
445,Yārāda,0,IN,1639975176,61,17.6500,83.2667,75.67,5.44,Ginger Vishakhapatnam Gajuwaka
547,Porto Velho,0,BR,1639975201,94,-8.7619,-63.9039,75.25,0.00,Hotel Slaviero Essential Porto Velho


In [9]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

7

In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Checking hotel info:
len(hotel_info)

7

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))